In [1]:
import pandas as pd
import numpy as np
import csv
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk import word_tokenize

In [2]:
## db에 넣은 다음 수정할 부분

data = pd.read_csv('E:/final_ing/final_ing/crawling/crawling_200407.csv')
ingre = data['ingre_main_oneline']
ingre = np.array(ingre.tolist())

In [3]:
with open('hv.pkl', 'rb') as f:
    X = pickle.load(f)
vectorize = HashingVectorizer(
    tokenizer = lambda x:x.split()
)

In [10]:
def recommend(ingre_input,main):

    srch_vector = vectorize.transform([ingre_input])
    cosine_similar = linear_kernel(srch_vector, X).flatten()
    rank_idx = cosine_similar.argsort()[::-1]
    count = 0

    idx_filtering=[]
    for i in rank_idx:
        if cosine_similar[i] > 0:
            if main in ingre[i]:
                #ingre_for_cv.append(ingre[i])
                idx_filtering.append(i)
                count+=1
                if count>100:
                    break
                    
    df = pd.DataFrame(ingre[idx_filtering],columns=['ingre'])
    df['idx_filtering']=idx_filtering
    df['calc'] = 0.
  
    ingre_for_cv = df['ingre'].tolist()
    ingre_for_cv.append(ingre_input)
    
    vect = CountVectorizer(min_df=0,tokenizer=lambda x:x.split())
    vect.fit(ingre_for_cv)
    cv=vect.transform(ingre_for_cv).toarray()
#     print(vect.get_feature_names())
    for idx, val in enumerate(cv[0:-1]):
        df['calc'][idx]=(val*cv[-1]).sum()/val.sum()
        print(idx, (val*cv[-1]).sum()/val.sum())
        
    df = df.sort_values(by=['calc'], axis=0, ascending=False)
    df = df.reset_index(drop=True)

    
    return df
  
  #  calc_idx = cal.argsort()[::-1]

In [27]:
df=recommend('대파 양파 돼지고기 계란 고추','돼지고기')
# df=recommend('양파 소고기 라면 계란','')

['中', '가지', '간거', '간장', '감자', '계란', '고추', '고추가루', '고추장', '공기밥', '김치', '깍뚜기김치', '깻잎', '꽈리고추', '냉이', '느타리버섯', '다시팩', '다진', '다진마늘', '다짐육', '달래', '당근', '대파', '돼지고기', '두부', '뒷다리', '뒷다리불고기용', '마늘', '만능간장', '면수', '목살', '묵은지', '물', '밥', '배추김치', '버섯', '부추', '부침가루', '불고기감', '불고기용', '새송이버섯', '설탕', '순두부', '스파게티면', '시금치', '식빵', '식용유', '신김치', '안심', '앞다리살', '애호박', '양배추', '양파', '오징어', '월계수잎', '육수', '작은거', '작은것', '잘게쓴김치', '잡채용', '전지', '제육', '주물럭', '중', '짜장라면', '참기름', '청', '청경채', '청국장', '청량고추', '청양고추', '춘장', '콩나물', '통깨', '통마늘', '파프리카', '팽이버섯', '표고버섯', '호박', '홍', '홍고추']
0 0.8
1 0.8
2 0.8
3 1.0
4 1.0
5 1.0
6 1.0
7 1.0
8 1.0
9 1.0
10 1.0
11 1.0
12 1.0
13 1.0
14 1.0
15 0.6666666666666666
16 0.6666666666666666
17 0.6666666666666666
18 0.5714285714285714
19 0.5714285714285714
20 0.5714285714285714
21 0.5714285714285714
22 0.5714285714285714
23 0.5714285714285714
24 0.5714285714285714
25 0.5714285714285714
26 0.5555555555555556
27 0.75
28 0.75
29 0.75
30 0.75
31 0.75
32 0.75
33 0.75
34 0.75
35 0.75
36 0.75
37 

C:\coding\anaconda\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [15]:
df

,ingre,idx_filtering,calc
0,돼지고기 대파,12248,1.000000
1,돼지고기 대파 양파,37227,1.000000
2,대파 돼지고기,5799,1.000000
3,돼지고기 대파,31571,1.000000
4,돼지고기 대파,20485,1.000000
...,...,...,...
96,돼지고기 불고기용 양파 대파 청 고추 홍 고추 깻잎,8459,0.555556
97,돼지고기 만능간장 고추장 고추가루 양파 대파 고추 참기름,95273,0.500000
98,가지 돼지고기 다짐육 양파 마늘 고추 대파 스파게티면,13874,0.500000
99,돼지고기 안심 계란 양파 작은것 통마늘 대파 월계수잎,7171,0.500000


In [28]:
df.iloc[0:20]

,ingre,idx_filtering,calc
0,돼지고기 대파,12248,1.0
1,돼지고기 대파 양파,37227,1.0
2,대파 돼지고기,5799,1.0
3,돼지고기 대파,31571,1.0
4,돼지고기 대파,20485,1.0
5,돼지고기 양파 대파,10133,1.0
6,돼지고기 계란 대파,235,1.0
7,돼지고기 양파 대파,4114,1.0
8,돼지고기 양파 대파,11057,1.0
9,돼지고기 양파 대파,2888,1.0


In [18]:
test = data.iloc[df['idx_filtering'][0]]

In [19]:
test

id                                                              6909298
title                                                     돼지고기 장조림~*^^*
writer                                                              나야.
viewers                                                            3877
size                                                                  6
                                            ...                        
steps                 ['물을 끓여준비합니다', '돼지고기 안심\n600g을 \n먹기좋은 크기로 썰어 준...
steps_img             ['https://recipe1.ezmember.co.kr/cache/recipe/...
ingredients_tagged       ['돼지고기', '대파', '간장', '맛술', '설탕', '올리고당', '생강']
main_tagged                                              ['돼지고기', '대파']
minor_tagged                                                         []
Name: 20485, Length: 73, dtype: object

In [181]:
for i in data.iloc[df['idx_filtering'][0]][0:-6]:
    if pd.notnull(i):
        print(i)

6853440
아이 반찬 - 초간단 규동 만들기
불량엄마소자매마미
7974.0
1.0
15.0
아무나
['소고기', '양파', '계란', '쯔유', '맛술', '물']
아이밥 매끼니 반찬이 걱정일때~후다닥 만들어주면 좋은 규동이예요. 재료도 초간단 15분 레시피랍니다~^^ 아이밥으로 만든거라 양이 작아요~어른들이 드실때는 양만 늘려주시면 좋을듯^^                
['100g', '1/4개', '1개', '2수저', '2수저', '3수저']
['소고기', '양파', '계란']
['100g', '1/4개', '1개']
[]
[]
['쯔유', '맛술', '물']
['2수저', '2수저', '3수저']
양파는 얇게 썰어주세요~!
https://recipe1.ezmember.co.kr/cache/recipe/2016/07/26/a1cfb8d0e8cac70f2bc5ae7a8e396b041.jpg
쯔유 2수저, 맛술 2수저, 물 3수저를 넣고
https://recipe1.ezmember.co.kr/cache/recipe/2016/07/26/984d7058506567fddf1dda969a291bae1.jpg
양파를 넣어 후루룩~끓여주세요~!
https://recipe1.ezmember.co.kr/cache/recipe/2016/07/26/2fb1e1bbdae50e940d2a01eb5555ae971.jpg
양파가 투명해지면 불을 끄세요.
https://recipe1.ezmember.co.kr/cache/recipe/2016/07/26/0eaace12a13018e999efa8c0773831c71.jpg
질깃한 식감의 고기는 안 먹고 뱉는 큰딸 때문에~
혹시나 싶어~
소고기에 칼집을 넣어주었어요.
https://recipe1.ezmember.co.kr/cache/recipe/2016/07/26/3d7110216ace62b0628cc66906e68ecd1.jpg
앞뒤로 살짝만 팬에 구워주세요~!
https://recipe1.ezmember.co.kr/cache/rec